In [1]:
import pandas as pd
import datetime
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px

In [2]:
file = pd.read_excel("labeled_al.xlsx")

In [3]:
def to_str(v):
    return str(int(float(v)))
def to_int(v):
    return int(float(v))

In [4]:
file.columns

Index(['                    tweet_id', '                tweet_id', 'date',
       'hts', 'is_quote', 'is_retweet', 'lat', 'likes', 'long', 'mentions',
       'original_tweet_id', 'quote_count', 'quoted_text', 'replies',
       'retweets', 'text', 'tweet_id', 'user', 'label', 'full_text',
       'all_tweet_id', 'original_label'],
      dtype='object')

In [5]:
file.label.value_counts()

2    824
1    116
3     15
Name: label, dtype: int64

In [6]:
# file["tweet_id"] = file["tweet_id"].fillna(file["                    tweet_id"]).fillna(file["                tweet_id"])

In [7]:
# file.drop(columns=["                    tweet_id","                tweet_id"],inplace = True)

In [8]:
file["all_tweet_id"] = file["all_tweet_id"].map(to_int)
file["original_tweet_id"] = file["original_tweet_id"].map(to_int)

In [9]:
file.head()

,tweet_id,tweet_id,date,hts,is_quote,is_retweet,lat,likes,long,mentions,...,quoted_text,replies,retweets,text,tweet_id,user,label,full_text,all_tweet_id,original_label
0,1.244995e+18,NaN,2020-03-31 14:28:38,Coronavirustruth;covid19zambia,False,True,NaN,0,NaN,NaN,...,NaN,0,0,RT @ZambiaAsks: Is spraying alcohol or chlorin...,NaN,OpenParlyZED,3,RT @ZambiaAsks: Is spraying alcohol or chlorin...,1244995022040290048,3
1,1.245090e+18,NaN,2020-03-31 20:47:23,NaN,False,True,NaN,0,NaN,NaN,...,NaN,0,0,RT @180nobs1: ÃƒÂ¢Ã‚Â€Ã‚Â˜DonÃƒÂ¢Ã‚Â€Ã‚Â™t let...,NaN,adejohenemosah1,2,RT @180nobs1: ÃƒÂ¢Ã‚Â€Ã‚Â˜DonÃƒÂ¢Ã‚Â€Ã‚Â™t let...,1245090338156539904,2
2,1.245353e+18,NaN,2020-04-01 14:10:11,NaN,False,True,NaN,0,NaN,NaN,...,NaN,0,0,RT @GBDave: Breaking News: alcohol consumption...,NaN,breakingnewsbo1,2,RT @GBDave: Breaking News: alcohol consumption...,1245352766815460096,2
3,1.245483e+18,NaN,2020-04-01 22:46:44,NaN,False,True,NaN,0,NaN,NaN,...,NaN,0,0,RT @billyjack2016: For those idiots who think ...,NaN,Gamble23Daisy,2,RT @billyjack2016: For those idiots who think ...,1245482761525149952,2
4,1.245558e+18,NaN,2020-04-02 03:45:34,NaN,False,True,NaN,0,NaN,NaN,...,NaN,0,0,RT @MadamEarth: German health expert claims dr...,NaN,Meggles22Megan,1,RT @MadamEarth: German health expert claims dr...,1245557964661900032,1


In [10]:
file.shape

(955, 22)

In [11]:
day_list = []
for i in range(file.shape[0]):
    if(file.iloc[i]["date"] == file.iloc[i]["date"]):
        #print(str(file.iloc[i]["date"].date()))
        day = str(file.iloc[i]["date"].date())
        day_list.append(day)
    else:
        day_list.append("0")
file["day"] = day_list

In [12]:
days = file["day"].unique()
dates = [datetime.datetime.strptime(ts, "%Y-%m-%d") for ts in days]
dates.sort()
sorteddates = [datetime.datetime.strftime(ts, "%Y-%m-%d") for ts in dates]

In [13]:
G = nx.Graph()

for i in range(file.shape[0]):
    node1 = int(file[file.index==i]["all_tweet_id"].values[0])
    node1_label = int(file[file.index==i]["label"].values[0])
    node2 = int(file[file.index==i]["original_tweet_id"].values[0])
    node2_label = int(file[file.index==i]["original_label"].values[0])
    date = file[file.index==i]["day"].values[0]
    G.add_node(node1, cat=node1_label, day=date)
    G.add_node(node2, cat=node2_label, day=date)
    G.add_edge(node1, node2)

In [14]:
G.number_of_nodes(), G.number_of_edges()

(1055, 955)

In [15]:
# store subgraphs in graph_dict = {date:subgraph}
graph_dict = {}
node_day_cum = []
i = 0
for day in sorteddates:
    node_day = [x for x,y in G.nodes(data=True) if y['day']==day]
    node_day_cum += node_day
    G_sub = G.subgraph(node_day_cum)
    graph_dict[day] = G_sub
    i += 1

In [40]:
def showGraph(G, date):
    pos = nx.spring_layout(G)

    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='text',
        mode='lines',
        marker = dict())

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_adjacencies = []
    node_text = []
    node_cat = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_cat.append(G.nodes[adjacencies[0]]['cat'])
        node_text.append('\n # of connections: '+str(len(adjacencies[1])))

    node_trace.marker.color = node_cat
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                 layout=go.Layout(
                    title=date + "                        number of tweets: " + str(G.number_of_nodes()) + "                         "+\
                     str(len([n for n,v in G.nodes(data=True) if v['cat'] == 1])) + " supportive, " + \
                     str(len([n for n,v in G.nodes(data=True) if v['cat'] == 2])) + " denial, " + \
                     str(len([n for n,v in G.nodes(data=True) if v['cat'] == 3])) + " neutral ",
                    titlefont_size=16,
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text="",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )
    fig.show()

In [25]:
graph_dict.keys()

dict_keys(['2020-03-11', '2020-03-12', '2020-03-13', '2020-03-14', '2020-03-15', '2020-03-16', '2020-03-17', '2020-03-18', '2020-03-25', '2020-03-27', '2020-03-28', '2020-03-31', '2020-04-01', '2020-04-02', '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-06', '2020-04-07', '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-11', '2020-04-12', '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-24', '2020-04-25', '2020-04-26', '2020-04-27', '2020-04-28', '2020-04-30', '2020-05-01', '2020-05-02'])

In [50]:
date = '2020-05-02'
H = graph_dict[date]
H_unfrozen = nx.Graph(H)
H_unfrozen.remove_nodes_from([node for node,degree in dict(H.degree()).items() if degree == 0])
showGraph(H_unfrozen, date)

In [76]:
import collections
degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
degreeCount = collections.Counter(degree_sequence)
deg, cnt = zip(*degreeCount.items())
sum_degree = sum(cnt)
#cnt = tuple(t/sum_degree for t in cnt)
deg, cnt

((379, 80, 67, 55, 29, 26, 18, 15, 13, 11, 9, 8, 6, 5, 4, 3, 2, 1),
 (1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 4, 3, 5, 8, 6, 17, 997))

In [64]:
fig = px.scatter(x=deg , y=cnt, log_x=True, log_y=True, labels={"x":"Degree K", "y":"Probability p(k)"}, title="node degree log-log plot for rumor alcohol") 
fig.show()

In [19]:
day_list = []
cat_1_list = []
cat_2_list = []
cat_3_list = []

for day in graph_dict:
    cat_list = list(nx.get_node_attributes(graph_dict[day], name="cat").values())
    num_1 = cat_list.count(1)
    num_2 = cat_list.count(2)
    num_3 = cat_list.count(3)
    
    day_list.append(day)
    cat_1_list.append(num_1)
    cat_2_list.append(num_2)
    cat_3_list.append(num_3)

fig = go.Figure()
fig.add_trace(go.Scatter(x=day_list, y=cat_1_list,
                    mode='lines',
                    name='supportive',
                    line_shape='spline'))
fig.add_trace(go.Scatter(x=day_list, y=cat_2_list,
                    mode='lines',
                    name='denial',
                    line_shape='spline'))
fig.add_trace(go.Scatter(x=day_list, y=cat_3_list,
                    mode='lines', 
                    name='neutral',
                    line_shape='spline'))

fig.show()

In [75]:
dc = nx.degree_centrality(H)
sort_orders = sorted(dc.items(), key=lambda x: x[1], reverse=True)
sort_orders[0:10]

[(1247721359108709888, 0.3595825426944972),
 (1252112258811000064, 0.07590132827324478),
 (1256249520284760064, 0.0635673624288425),
 (1255109780395490048, 0.05218216318785579),
 (1251164844880989952, 0.027514231499051234),
 (1250578857784199936, 0.024667931688804556),
 (1237460454588460032, 0.017077798861480076),
 (1254846183584890112, 0.014231499051233398),
 (1245533554005509888, 0.012333965844402278),
 (1246359997442899968, 0.012333965844402278)]

In [26]:
file[file["original_tweet_id"]==1254846183584890112]["original_label"]

735    2
736    2
737    2
738    2
739    2
740    2
741    2
742    2
743    2
744    2
745    2
746    2
747    2
750    2
764    2
Name: original_label, dtype: int64